![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © 2016 Carlos A. Iglesias

## [Introduction to Machine Learning II](3_0_0_Intro_ML_2.ipynb)

# Exercise - The Titanic Dataset

In this exercise we are going to put in practice what we have learnt in the notebooks of the session. 

Answer directly in your copy of the exercise and submit it as a moodle task.

In [2]:
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set(color_codes=True)

# if matplotlib is not set inline, you will not see plots
%matplotlib inline

# Reading Data

Assign the variable *df* a Dataframe with the Titanic Dataset from the URL https://raw.githubusercontent.com/gsi-upm/sitc/master/ml2/data-titanic/train.csv"

Print *df*.

In [3]:
url="https://raw.githubusercontent.com/gsi-upm/sitc/master/ml2/data-titanic/train.csv"
df = pd.read_csv(url)
df_original = df.copy() # Copy to have a version of df without modifications
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Munging and Exploratory visualisation

Obtain number of passengers and features of the dataset

Obtain general statistics (count, mean, std, min, max, 25%, 50%, 75%) about the column Age

Obtain the median of the age of the passengers

Obtain number of missing values per feature

How many passsengers have survived? List them grouped by Sex and Pclass.

Assign the result to a variable df_1 and print it

Visualise df_1 as an histogram.

# Feature Engineering

Here you can find some features that have been proposed for this dataset. Your task is to analyse them and provide some insights. 

Use pandas and visualisation to justify your conclusions

## Feature FamilySize 

Regarding SbSp and Parch, we can define a new feature, 'FamilySize' that is the combination of both.

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch']
df.head()

## Feature Alone

It seems many people who went alone survived. We can define a new feature 'Alone'

In [ ]:
df['Alone'] = (df.FamilySize == 0)
df.head()

## Feature Salutation

If we observe well in the name variable, there is a 'title' (Mr., Miss., Mrs.). We can add a feature wit this title.

In [ ]:
#Taken from http://www.analyticsvidhya.com/blog/2014/09/data-munging-python-using-pandas-baby-steps-python/
def name_extract(word):
    return word.split(',')[1].split('.')[0].strip()

df['Salutation'] = df['Name'].apply(name_extract)
df.head()

We can list the different salutations.

In [ ]:
df['Salutation'].unique()

In [ ]:
df.groupby(['Salutation']).size()

There only 4 main salutations, so we combine the rest of salutations in 'Others'.

In [ ]:
def group_salutation(old_salutation):
    if old_salutation == 'Mr':
        return('Mr')
    else:
        if old_salutation == 'Mrs':
            return('Mrs')
        else:
            if old_salutation == 'Master':
                return('Master')
            else: 
                if old_salutation == 'Miss':
                    return('Miss')
                else:
                    return('Others')
df['Salutation'] = df['Salutation'].apply(group_salutation)
df.groupby(['Salutation']).size()

In [ ]:
# Distribution
colors_sex = ['#ff69b4', 'b', 'r', 'y', 'm', 'c']
df.groupby('Salutation').size().plot(kind='bar', color=colors_sex)

In [ ]:
df.boxplot(column='Age', by = 'Salutation', sym='k.')

## Features Children and Female

In [ ]:
# Specific features for Children and Female since there are more survivors
df['Children']   = df['Age'].map(lambda x: 1 if x < 6.0 else 0)
df['Female']     = df['Sex'].map(lambda x: 1 if x == 0 else 0)

## Feature AgeGroup

In [ ]:
# Group ages to simplify machine learning algorithms.  0: 0-5, 1: 6-10, 2: 11-15, 3: 16-59 and 4: 60-80
df['AgeGroup'] = 0
df.loc[(.Age<6),'AgeGroup'] = 0
df.loc[(df.Age>=6) & (df.Age < 11),'AgeGroup'] = 1
df.loc[(df.Age>=11) & (df.Age < 16),'AgeGroup'] = 2
df.loc[(df.Age>=16) & (df.Age < 60),'AgeGroup'] = 3
df.loc[(df.Age>=60),'AgeGroup'] = 4

## Feature Deck
Only 1st class passengers have cabins, the rest are ‘Unknown’. A cabin number looks like ‘C123’. The letter refers to the deck.

In [7]:
def substrings_in_string(big_string, substrings):
    if type(big_string) == float:
        if np.isnan(big_string):
            return 'X'
    for substring in substrings:
        if big_string.find(substring) != 1:
            return substring
    print(big_string)
    return 'X'
 
#Turning cabin number into Deck
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']
df['Deck']=df['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))
df['Deck']


0      X
1      A
2      X
3      A
4      X
5      X
6      A
7      X
8      X
9      X
10     A
11     A
12     X
13     X
14     X
15     X
16     X
17     X
18     X
19     X
20     X
21     A
22     X
23     A
24     X
25     X
26     X
27     A
28     X
29     X
      ..
861    X
862    A
863    X
864    X
865    X
866    X
867    A
868    X
869    X
870    X
871    A
872    A
873    X
874    X
875    X
876    X
877    X
878    X
879    A
880    X
881    X
882    X
883    X
884    X
885    X
886    X
887    A
888    X
889    A
890    X
Name: Deck, dtype: object

## Feature FarePerPerson

This feature is created from two previous features: Fare and FamilySize.

In [ ]:
df['FarePerPerson']= df['Fare'] / (df['FamilySize'] + 1)

## Feature AgeClass

Since age and class are both numbers we can just multiply them and get a new feature.


In [ ]:
df['AgeClass']=df['Age']*df['Pclass']

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© 2016 Carlos A. Iglesias, Universidad Politécnica de Madrid.